# packages

In [129]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")

# imports

In [5]:
import pandas as pd
import numpy as np
import json
import csv
import os

# Read Extraction Data from CSV

In [44]:
df = pd.read_csv(filepath_or_buffer="data/extraction consolidation results_V0_3.csv",sep=";", encoding="latin-1")

In [45]:
result = df['Title'].to_list()

In [46]:
result = list(dict.fromkeys(result))
print(len(result))

29


In [47]:
titles = list(df.columns)

In [48]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [49]:
metadata = metadata.drop_duplicates(subset=['Title'])

In [50]:
data = df.merge(metadata, how='left', on='Title')
#unique function

In [51]:
del metadata
del df
del titles

In [52]:
index = 0
for row in data:
    print(index, row)
    index += 1

0 Title
1 DT Definition
2 virtual space
3 Kritzinger
4 DT is specific
5 modeling language
6 note on modeling language
7 model type
8 model processing technique
9 model usage technique
10 source
11 target
12 purpose of MDE application
13 purpose of DT
14 DTPurposeCategory
15 use case domain
16 system lifecycle phase
17 twinning target
18 twin lifecycle phase
19 purpose /expected benefit of using models
20 open challenges
21 Technological Readiness Level
22 notes
23 Use Case
24 Publication Title
25 Authors
26 Publication Year
27 Document Identifier
28 Publisher


# create JSON from CSV

In [53]:
modelTrafos = 'modelTrafos'

In [54]:
out_menge_1 = {}
for key in result:
    out_menge_1[key] = {'DT_Definition':None,
                        modelTrafos:{}, 
                        'open challenges':None,
                        'Technological Readiness Level':None, 
                        'notes':None, 
                        'Use Case':None, 
                        'twinning target': None, 
                        'use case domain':None,
                        'virtual space':None,
                        'Kritzinger':None,
                        'DT is specific':None,
                        'modeling language':[],
                        'purpose of DT':None,
                        'DTPurposeCategory':None
                        }

In [55]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()
            
            if row['DT Definition'] is not None and not pd.isna(row['DT Definition']):
                out_menge_1[title]['DT_Definition'] = row['DT Definition']

            if row['purpose of DT'] is not None and not pd.isna(row['purpose of DT']):
                out_menge_1[title]['purpose of DT'] = row['purpose of DT']

            if row['DTPurposeCategory'] is not None and not pd.isna(row['DTPurposeCategory']):
                out_menge_1[title]['DTPurposeCategory'] = row['DTPurposeCategory']

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_1[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_1[title]['twinning target'] is None:
                    out_menge_1[title]['twinning target'] = row['twinning target']
                else:
                    out_menge_1[title]['twinning target'] += ";"
                    out_menge_1[title]['twinning target'] +=  row['twinning target']

            if row['purpose /expected benefit of using models'] is not None and not pd.isna(row['purpose /expected benefit of using models']):
                out_menge_1[title]['purpose /expected benefit of using models'] = row['purpose /expected benefit of using models']

            if row['open challenges'] is not None and not pd.isna(row['open challenges']):
                out_menge_1[title]['open challenges'] = row['open challenges']

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_1[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['notes'] is not None and not pd.isna(row['notes']):
                out_menge_1[title]['notes'] = row['notes']

            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_1[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_1[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_1[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_1[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_1[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_1[title]['Publisher'] = row['Publisher']
            if row['DT Definition'] is not None and pd.isna(row['DT Definition']):
                if row['virtual space'] is not None and not pd.isna(row['virtual space']):
                    out_menge_1[title]['virtual space'] = row['virtual space']
            
                if row['Kritzinger'] is not None and not pd.isna(row['Kritzinger']):
                    out_menge_1[title]['Kritzinger'] = row['Kritzinger']
            
                if row['DT is specific'] is not None and not pd.isna(row['DT is specific']):
                    out_menge_1[title]['DT is specific'] = row['DT is specific']
            
            if row['modeling language'] is not None and not pd.isna(row['modeling language']):
                out_menge_1[title]['modeling language'].append(row['modeling language'])

            # multiple Trafos
            arrayindexes = [0,1,2,3,4,5,6,13,10+4,11+4,13+4,15+4,16+4,17+4,18+4,19+4,20+4,21+4,22+4,23+4,24+4]
            rowjson = rowcopy.drop(row.iloc[arrayindexes].index).to_json()
            out_menge_1[title][modelTrafos][rownum] = rowjson
            #out_menge_1[title]['data'][rownum].append(rowjson)
            rownum+=1


In [56]:
del rowjson
del rowcopy
del result
del rownum

# export JSON to /json/data_consolidated_semistructured.json

In [40]:
json_object = json.dumps(out_menge_1, indent = 4) 
try:
    os.mkdir('target/')
    os.mkdir('target/img/')
except OSError as error:
    print(error)
with open("target/json/data_consolidated_semistructured.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'target/'
